In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/ab_data.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_11_Estadistica_Inferencial/Data/ab_data.csv?token=AA4GFHLD7JL37UB2Z4P6O7C6YV4FE
    ! wget -O ../Data/klout-scores.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_11_Estadistica_Inferencial/Data/klout-scores.csv?token=AA4GFHKHCDGQ3VHY6XMID6K6YV4TQ
    ! wget -O ../Data/wildlife.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_11_Estadistica_Inferencial/Data/wildlife.csv?token=AA4GFHM5EEHYTE5ARKPTUHS6YV4WM    
        
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

## Imports

In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import math
import datetime


---

## Intervalos de Confianza

### Dataset

Utilizamos una muestra aleatoria de puntajes de influencia de redes sociales del servicio http://klout.com. 

Klout era un sitio web y una aplicación móvil que utilizaba análisis de redes sociales para calificar usuarios de acuerdo con su influencia social en línea. 

El sitio calculaba un "Klout Score", que era un valor numérico entre 1 y 100 donde puntuaciones más altas correspondian a una mayor "influencia social" en línea.

De una población de más de 620 millones de puntajes obtuvimos una muestra de tamaño 1048

### Leemos los datos

In [ ]:
data = pd.read_csv('../Data/klout-scores.csv', header=None, names=['scores'])
print(data.shape)
data.head()

### Exploratorio

Grafiquemos los valores de score que leimos en la variable data, y observemos que hay dos picos. Uno corresponde a los consumidores y el otro a los influencers.

In [ ]:
p=sns.distplot(data.scores)

### Intervalo de confianza

Si queremos estimar un parámetro poblacional (media, proporción, desvío estandar) a partir de un estadístico muestral, no podemos estar seguros del resultado pero podemos dar algún nivel de confianza a nuestra predicción por medio de un intervalo de confianza (CI).

|   |media|proporción|desvío estandar|   |
|---|:---:|:---:|:---:|---|
|población|$\mu$|$p$|$\sigma$|parámetros|
|muestra|$\bar{x}$|$\hat{p}$|$s$|estadísticos|

Calculemos un intervalo de confianza del 95% para la media muestral del dataset Klout Scores.

Para muestras grandes, podemos resolver esta ecuación con un nivel de alfa de $\alpha=.05$

$$\bar{x}+z_{\alpha/2}\cdot\frac{\sigma}{\sqrt{n}}\lt\mu_{estimator}\lt\bar{x}-z_{\alpha/2}\cdot\frac{\sigma}{\sqrt{n}}$$ 

Calculemos estos valores para nuestros datos:

#### Media muestral:

#### Desvío estandar de la muestra

Como nuestar muestra es grande (de tamaño mayor que 30) podemos usar el desvío estandar muestral como aproximación de sigma


#### Error estandar de la muestra:


#### z-scores:

Calculamos los z-score para poder calcular los límites inferior y superior del intervalo de confianza.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

**CDF**: Cumulative Distribution Function usamos `stats.cdf`

**ppf**: Percent Point Function (Inverse of CDF) usamos `stats.ppf`

In [ ]:
alpha = 0.05

Una forma alternativa para obtener el intervalo de valores críticos es usar 

`interval(alpha, loc=0, scale=1)` que devuelve los límites del rango que contiene alfa-porciento de la distribución

Observemos que el argumento alfa de esta función **no** es el valor de alfa que definimos arriba, sino 1 - ese valor. 

Llamaremos nivel de confianza (confidence coefficient) al valor del argumento alpha de la función interval.

https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.norm.html

In [ ]:
confidence_coef = 1 - alpha


#### Calculamos los límites inferior y superior del intervalo de confianza para la media de Klout Score:

#### ¿Qué significa este resultado?

Un intervalo de confianza trata de capturar la media de la población real de una muestra declarando un intervalo de confianza. Esto significa que el 95% de los intervalos que calculemos a partir de muestras independientes atrapan el verdadero valor de la media poblacional.

Klout.com afirma que su puntaje promedio es 40, por lo que no obtuvimos el parámetro promedio de población real (el valor que obtuvimos es 37.7). Dado que nuestros cálculos parecen ser correctos, esto podría significar que:
* Tal vez nuestra media muestral está muy por debajo de la media poblacional. Tengamos en cuenta que solo tenemos un 95% de confianza.
* Quizás la comunicación de Klout sobre el puntaje promedio se simplifica a un valor fácil de recordar de 40.


#### Efecto del tamaño de muestra

El tamaño de la muestra afecta los límites del intervalo de confianza. Cuanto más pequeña es la muestra, menos confianza tenemos, por lo tanto, más amplio es el intervalo de confianza. Probemos esto con una muestra aleatoria de los datos de Klout Score de n = 50.

Generamos la muestra.

Graficamos la distribución de la muestra

Recalculamos el intervalo de confianza:

Con este tamaño de muestra, estamos 95% seguros de que atrapamos el verdadero valor de la muestra en el intervalo klout_sample_CI_mean_lower, klout_sample_CI_mean_upper. 

Aunque este intervalo de confianza también capta el parámetro media, tiene un rango mucho más grande.

## Ejercicio

Hacer una función que dados
* una población como una instancia de Series 
* un tamaño de muestra 
* un valor de alfa
* un valor de sigma
Calcule el tamaño del intervarlo de confianza para la media con ese alfa, como máximo - mínimo.

Para 20 valores de tamaño de muestra, grafiquemos el tam del intervalo vs tamaño de muestra

Ayuda: https://numpy.org/doc/stable/reference/generated/numpy.linspace.html

---

## Test de Hipótesis

### Data

El dataset que usamos en esta parte contiene reportes de  [Federal Aviation Administration Wildlife Strike Database](http://wildlife.faa.gov/database.aspx) correspondientes a los años 2012 y 2013 en el estado de California, USA.

Usaremos los datos diarios de frecuencia de incidentes de golpes a fauna silvestre.

### Leemos los datos

In [ ]:
# Load the data from a csv file. 
data_location = "../Data/wildlife.csv"
data = pd.read_csv(data_location)
data.head()


### Ejercicio: Preparación de los datos

Construyamos un dataset que tenga como columnas

* la fecha del incidente, de tipo datetime

Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

* la cantidad de incidentes en esa fecha, de tipo int 

Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html


### Ejercicio: Preparación de los datos - continuación

Queremos que el DataFrame que creamos en el paso anterior tenga un registro por cada día del año 2012 y 2013.

Para eso vamos a: 

1) Asignar el valor del campo INCIDENT_DATE como índice del DataFrame construído en el punto anterior 

2) Crear un nuevo DataFrame que tenga sólo un índice y ninguna columna, que sea todas las fechas existentes durante los años 2012 y 2013 

Ayuda: `pandas.date_range` https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html

3) Hacer un join entre los DataFrame 1) y 2). Con esto vamos a conseguir que en el DataFrame resultado haya valores null en el campo INCIDENT_COUNT para las fechas que no estaban en el DataFrame resultado del ejercicio anterior.

Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

4) Por último, completamos los valores nulos de INCIDENT_COUNT en el nuevo DataFrame con el valor cero

Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html


### Ejercicio: Exploratorio

Usemos un gráficos de barras para representar los valores de frecuencia de accidentes en los años 2012 y 2013 (por separado)

Para eso, agregamos al DataFrame una columna de tipo int que indique el mes que corresponde al valor de index de cada registro.

Y usemos los valores de esta nueva columna como eje x.

In [ ]:
# magia!
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-label
data_2012 = data_2012_2013['2012']
data_2013 = data_2012_2013['2013']


### Ejercicio: Test de hipótesis

Asumamos que la Federal Aviation Administration lanzó en el 2013 un nuevo programa de prevención de incidentes con fauna silvestre.

Queremos saber si hay una baja significativa en el número diario de incidentes del año 2013 respecto del 2012.

Elegimos como nivel de significación (alfa) 0.05

Una probabilidad menor a 0.05 rechaza la hipótesis nula.

La hipótesis nula es que la media de incidentes del 2012 es la media poblacional y es igual a la media poblacional de incidentes del 2013.

La hipótesis alternativa es que la media de incidentes del 2013 es menor que la del 2012.

|Hypothesis|$\alpha = .05$|   |
|---|:---:|:---|
|Null|$H_0:$|$\mu = \bar{x}_{2013}$|
|Alternative|$H_a:$|$\mu \gt \bar{x}_{2013} $|

Calculemos la media de incidentes y desvío para los años 2012 y 2013

Notamos que la media de incidentes del 2013 es un poco menor que la del 2102.

Queremos saber si esta diferencia se debe a la variación normal de estos dato, es decir que la diferencia se puede adjudicar al azar. 

Para eso, calculemos el z-score y usemos un nivel de significación 0.05



Calculamos valores críticos:

Grafiquemos datos con distribución normal, la región de rechazo, y el valor de zscore obtenido

(Nota: Hay que modificar este código para que tome las variables a las que ustedes asignaron los resultados)

In [ ]:
# Plot the normal distribution
samples = 100
x_plot = np.linspace(-3.5, 3.5, samples)
y_plot = stats.norm.pdf(x_plot, 0, 1)
plt.plot(x_plot, y_plot)

# Plot the critical region
x_crit = np.linspace(-3.5, critical_value, samples)
y_crit = stats.norm.pdf(x_crit, 0, 1)
# colorea la region de rechazo de H0:
plt.fill_between(x_crit,  y_crit, alpha=.5)

# Plot the z score, linea naranja:
plt.plot([zscore, zscore], [0, stats.norm.pdf(zscore)])

# Show legend
plt.legend(['critical region', 'z score'])
plt.show()

Como zscore no es menor que critical_value, no podemos rechazar H0.

Esto indica que podemos obtener por azar una media muestral con ese valor de la misma población real.

En otras palabras, no hay diferencia significativa en los promedios de incidentes en 2012 y 2013.


---

## A/B Testing

### Dataset

Los datos corresponden a las visitas de usuarios a un sitio web.

Este sitio tiene dos versiones de la landing: old_page y new_page

El campo "converted" indica si un usuario hizo click o no, idicados con 1 o 0 respectivamente, en la pagina que vio.

El objetivo es determinar si la nueva versión de la página tiene más proporción de clicks que la vieja.

https://www.kaggle.com/zhangluyuan/ab-testing

In [ ]:
data_location = "../Data/ab_data.csv"
data = pd.read_csv(data_location, sep=",")
data.head()

Tenemos que eliminar todos los registros que sean 
* 'control' y 'new_page' 
* 'treatment' y 'old_page'

Miremos cuántos no están en esas condiciones

Eliminemos los registros detectados en el paso anterior

Eliminemos los registros duplicados

Calculemos la probabilidad de conversion independiente de la página

Calculemos la probabilidad de convertir si el usuario ve la página nueva

In [ ]:
new_mask = data_ab.landing_page == 'new_page'
new_converted_mask = np.logical_and(new_mask, data_ab.converted == 1)
new_converted_mask.sum() / new_mask.sum()

Calculemos la probabilidad de convertir si si el usuario ve la página vieja

In [ ]:
old_mask = data_ab.landing_page == 'old_page'
old_converted_mask = np.logical_and(old_mask, data_ab.converted == 1)
old_converted_mask.sum() / old_mask.sum()

Calculemos la probabilidad de ver la página nueva

In [ ]:
new_mask.sum() / data_ab.shape[0]

Definimos una función que calcula los estimadores de los parámetros de una distribución de Bernoulli

In [ ]:
def estimated_parameters(N, n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p, sigma

Definimos una función que calcula el estadístico de un A/B Test

In [ ]:
def a_b_test_statistic(N_A,n_A,N_B,n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A)/math.sqrt(sigma_A**2 + sigma_B**2)

Calculemos el valor del estadístico definido por esta función y los valores críticos y p-value para decidir si rechazamos H0 con un nivel de significación del 5%.

H0: p_new = p_old

H1: p_new != p_old

**Bonus track**:
    
Más adelante en este curso vamos ver la biblioteca statsmodels, que ahora vamos a mencionar porque nos permite calcular z-score y p-value para nuestro problema, y compararlo con los resultados que obtuvimos con nuestras funciones.

https://www.statsmodels.org/0.6.1/generated/statsmodels.stats.proportion.proportions_ztest.html

In [ ]:
import statsmodels.api as sm

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([n_control_old, n_treatment_new], [N_control_old, N_treatment_new], 
                                              alternative='two-sided')
z_score, p_value

## Referencias


Intervalos de Confianza
https://github.com/leonvanbokhorst/NoteBooks-Statistics-and-MachineLearning/blob/master/0013%20Confidence%20Interval%20of%20a%20Klout%20Score%20sample.ipynb

Test de Hipótesis
https://github.com/leonvanbokhorst/NoteBooks-Statistics-and-MachineLearning/blob/master/0014%20Hypothesis%20Testing%20with%20Bird%20Strike%20Incidents.ipynb

A/B testing
https://www.kaggle.com/shweta112/a-b-testing-analysis

A/B testing, un ejemplo (un poco más difícil) para analizar 

https://www.kaggle.com/tammyrotem/ab-tests-with-python
    
https://github.com/baumanab/udacity_ABTesting#summary
        
https://github.com/TammyRotem/A-B_Tests_with_Python/blob/master/AB_Testing_with_Python.ipynb        
